# QC-Py-09 : Types d'Ordres et Order Management dans QuantConnect

**Duree estimee** : 75 minutes

**Objectifs d'apprentissage** :
1. Maitriser les ordres de base (Market, Limit) et leur utilisation
2. Comprendre les ordres stop (Stop Market, Stop Limit) pour la gestion du risque
3. Utiliser les ordres speciaux (MOO, MOC, LIT) pour des executions precises
4. Gerer les ordres avec les Order Tickets (modification, annulation, statuts)
5. Implementer des strategies avec bracket orders et trailing stops

**Prerequis** :
- Notebooks QC-Py-01 a QC-Py-08 completes
- Comprehension de base des marches financiers
- Notions de Python intermediaire

---
## Plan du notebook

1. **Ordres de Base** (25 min) - Market Orders, Limit Orders
2. **Ordres Stop** (20 min) - Stop Market, Stop Limit
3. **Ordres Speciaux** (20 min) - MOO, MOC, LIT
4. **Order Management** (25 min) - Tickets, Statuts, Events
5. **Combo Orders** (15 min) - Bracket Orders, Best Practices
6. **Strategie Complete** (20 min) - Breakout Strategy

---
## Vue d'ensemble des Types d'Ordres

| Type d'Ordre | Methode | Execution | Usage Principal |
|--------------|---------|-----------|------------------|
| **Market** | `MarketOrder()` | Immediate | Entrees/sorties urgentes |
| **Limit** | `LimitOrder()` | Si prix atteint | Acheter moins cher |
| **Stop Market** | `StopMarketOrder()` | Market si stop touche | Stop-loss |
| **Stop Limit** | `StopLimitOrder()` | Limit si stop touche | Stop avec controle |
| **MOO/MOC** | `MarketOnOpen/CloseOrder()` | Open/Close | Timing specifique |
| **LIT** | `LimitIfTouchedOrder()` | Limit si trigger | Pullback entry |

In [ ]:
# Reference rapide: Tous les types d'ordres
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class OrderTypesQuickReference(QCAlgorithm):
    """
    Reference rapide de tous les types d'ordres QuantConnect.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
    
    def ShowAllOrderTypes(self, price):
        """Demonstration de tous les types d'ordres."""
        
        # 1. MARKET ORDER - Execution immediate
        self.MarketOrder(self.spy, 100)  # Acheter
        self.MarketOrder(self.spy, -50)  # Vendre
        
        # 2. LIMIT ORDER - Prix specifique
        self.LimitOrder(self.spy, 100, price * 0.98)   # Acheter si prix <= 98%
        self.LimitOrder(self.spy, -100, price * 1.02)  # Vendre si prix >= 102%
        
        # 3. STOP MARKET ORDER - Declenche market order
        self.StopMarketOrder(self.spy, -100, price * 0.95)  # Stop-loss
        self.StopMarketOrder(self.spy, 100, price * 1.05)   # Breakout entry
        
        # 4. STOP LIMIT ORDER - Declenche limit order
        self.StopLimitOrder(self.spy, -100, price * 0.95, price * 0.94)
        
        # 5. MARKET ON OPEN/CLOSE
        self.MarketOnOpenOrder(self.spy, 100)   # A l'ouverture
        self.MarketOnCloseOrder(self.spy, -100) # A la cloture
        
        # 6. LIMIT IF TOUCHED
        self.LimitIfTouchedOrder(self.spy, 100, price * 0.98, price * 0.99)
        
        # 7. SET HOLDINGS (allocation par %)
        self.SetHoldings(self.spy, 0.5)   # 50% du portfolio
        self.SetHoldings(self.spy, -0.3)  # Short 30%

---
## PARTIE 1 : Ordres de Base (25 min)

### 1.1 Market Orders

Un **Market Order** s'execute immediatement au meilleur prix disponible.

| Avantage | Inconvenient |
|----------|---------------|
| Execution garantie | Pas de controle sur le prix |
| Rapidite | Slippage possible |

In [ ]:
# Exemple de Market Orders
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class MarketOrderExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.example_executed = False
    
    def OnData(self, data):
        if self.example_executed or not data.ContainsKey(self.spy):
            return
        
        price = data[self.spy].Close
        
        # MarketOrder: Acheter exactement 50 actions
        ticket = self.MarketOrder(self.spy, 50)
        self.Log(f"MarketOrder: Buy 50 SPY @ ~${price:.2f}")
        self.Log(f"Order ID: {ticket.OrderId}")
        
        self.example_executed = True
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            self.Log(f"[FILLED] {orderEvent.FillQuantity} @ ${orderEvent.FillPrice:.2f}")

In [ ]:
# SetHoldings: Allocation par pourcentage
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class SetHoldingsExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.qqq = self.AddEquity("QQQ", Resolution.Daily).Symbol
    
    def OnData(self, data):
        if self.Portfolio.Invested:
            return
        
        # Allouer 50% a SPY et 30% a QQQ
        self.SetHoldings(self.spy, 0.5)  # 50%
        self.SetHoldings(self.qqq, 0.3)  # 30%
        # 20% reste en cash
        
        self.Log(f"Portfolio allocated: SPY 50%, QQQ 30%, Cash 20%")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            symbol = orderEvent.Symbol.Value
            self.Log(f"[{symbol}] Filled: {orderEvent.FillQuantity} @ ${orderEvent.FillPrice:.2f}")

### 1.2 Limit Orders

Un **Limit Order** ne s'execute que si le prix atteint le niveau specifie.

| Direction | Limit Price | Execution |
|-----------|-------------|------------|
| **Achat** | Prix maximum | Si marche <= limit |
| **Vente** | Prix minimum | Si marche >= limit |

In [ ]:
# Exemple de Limit Orders
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class LimitOrderExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 3, 31)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.buy_ticket = None
        self.order_placed = False
    
    def OnData(self, data):
        if not data.ContainsKey(self.spy):
            return
        
        current_price = data[self.spy].Close
        
        # Placer un limit order une seule fois
        if not self.order_placed:
            limit_price = current_price * 0.99  # 1% sous le prix actuel
            
            self.buy_ticket = self.LimitOrder(self.spy, 100, limit_price)
            
            self.Log(f"Limit Order: Buy 100 @ ${limit_price:.2f}")
            self.Log(f"Current Price: ${current_price:.2f}")
            
            self.order_placed = True
        
        # Verifier le statut
        if self.buy_ticket:
            if self.buy_ticket.Status == OrderStatus.Filled:
                self.Log(f"Filled at ${self.buy_ticket.AverageFillPrice:.2f}")
    
    def OnOrderEvent(self, orderEvent):
        self.Log(f"Order {orderEvent.OrderId}: {orderEvent.Status}")

In [ ]:
# Statuts des ordres
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class OrderStatusExample(QCAlgorithm):
    """Demonstration des differents statuts d'ordre."""
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
    
    def OnOrderEvent(self, orderEvent):
        """Gerer tous les statuts possibles."""
        status = orderEvent.Status
        
        if status == OrderStatus.New:
            self.Log("Ordre cree")
        elif status == OrderStatus.Submitted:
            self.Log("Ordre soumis au marche")
        elif status == OrderStatus.PartiallyFilled:
            self.Log(f"Partiellement rempli: {orderEvent.FillQuantity}")
        elif status == OrderStatus.Filled:
            self.Log(f"Execute: {orderEvent.FillQuantity} @ ${orderEvent.FillPrice:.2f}")
        elif status == OrderStatus.Canceled:
            self.Log("Ordre annule")
        elif status == OrderStatus.Invalid:
            self.Log(f"Ordre invalide: {orderEvent.Message}")

---
## PARTIE 2 : Ordres Stop (20 min)

### 2.1 Stop Market Orders

Un **Stop Market Order** devient un market order quand le stop price est atteint.

| Direction | Stop Price | Declenchement |
|-----------|------------|----------------|
| **Vente (Stop-Loss)** | Sous le prix actuel | Si prix <= stop |
| **Achat (Breakout)** | Au-dessus du prix | Si prix >= stop |

In [ ]:
# Stop Market Order pour Stop-Loss
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class StopMarketExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.entry_price = None
        self.stop_ticket = None
        self.stop_pct = 0.05  # 5%
    
    def OnData(self, data):
        if not data.ContainsKey(self.spy):
            return
        
        price = data[self.spy].Close
        
        if not self.Portfolio.Invested:
            # Acheter
            self.MarketOrder(self.spy, 100)
            self.entry_price = price
            
            # Placer stop-loss a 5%
            stop_price = self.entry_price * (1 - self.stop_pct)
            self.stop_ticket = self.StopMarketOrder(self.spy, -100, stop_price)
            
            self.Log(f"Entry: ${self.entry_price:.2f}")
            self.Log(f"Stop-Loss: ${stop_price:.2f} (-{self.stop_pct*100}%)")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            if self.stop_ticket and orderEvent.OrderId == self.stop_ticket.OrderId:
                loss = (self.entry_price - orderEvent.FillPrice) * 100
                self.Log(f"[STOP-LOSS] Loss: ${loss:.2f}")

In [ ]:
# Stop Market Order pour Breakout Entry
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class BreakoutEntryExample(QCAlgorithm):
    """Entrer sur breakout au-dessus d'une resistance."""
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Resistance = plus haut 20 jours
        self.high_20 = self.MAX(self.spy, 20, Resolution.Daily)
        self.SetWarmUp(25)
        
        self.breakout_order = None
    
    def OnData(self, data):
        if self.IsWarmingUp or not self.high_20.IsReady:
            return
        
        if self.Portfolio.Invested or self.breakout_order:
            return
        
        resistance = self.high_20.Current.Value
        entry_price = resistance * 1.002  # 0.2% au-dessus
        
        # Stop order pour entrer sur breakout
        self.breakout_order = self.StopMarketOrder(self.spy, 100, entry_price)
        self.Log(f"Breakout entry placed at ${entry_price:.2f}")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            self.Log(f"[BREAKOUT] Entered at ${orderEvent.FillPrice:.2f}")

### 2.2 Stop Limit Orders

Un **Stop Limit Order** devient un limit order quand le stop est touche.

| Aspect | Stop Market | Stop Limit |
|--------|-------------|------------|
| Execution | Garantie | Non garantie |
| Controle prix | Non | Oui |
| Slippage | Possible | Limite |

In [ ]:
# Stop Limit Order
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class StopLimitExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.entry_price = None
        self.stop_limit_ticket = None
    
    def OnData(self, data):
        if not data.ContainsKey(self.spy):
            return
        
        if not self.Portfolio.Invested:
            price = data[self.spy].Close
            self.MarketOrder(self.spy, 100)
            self.entry_price = price
            
            # Stop-Limit: Stop a 5%, Limit a 5.5%
            stop_price = price * 0.95
            limit_price = price * 0.945
            
            self.stop_limit_ticket = self.StopLimitOrder(
                self.spy, -100, stop_price, limit_price
            )
            
            self.Log(f"Entry: ${price:.2f}")
            self.Log(f"Stop: ${stop_price:.2f}, Limit: ${limit_price:.2f}")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            if self.stop_limit_ticket and orderEvent.OrderId == self.stop_limit_ticket.OrderId:
                self.Log(f"[STOP-LIMIT] Executed at ${orderEvent.FillPrice:.2f}")

---
## PARTIE 3 : Ordres Speciaux (20 min)

### 3.1 Market On Open / Market On Close

| Type | Execution | Usage |
|------|-----------|-------|
| **MOO** | A l'ouverture | Strategies overnight |
| **MOC** | A la cloture | Rebalancement EOD |

In [ ]:
# Market On Open / Market On Close
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class MOOMOCExample(QCAlgorithm):
    """Acheter lundi a l'open, vendre vendredi au close."""
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Minute).Symbol
        
        # Scheduled events
        self.Schedule.On(
            self.DateRules.Every(DayOfWeek.Monday),
            self.TimeRules.BeforeMarketOpen(self.spy, 30),
            self.BuyAtOpen
        )
        
        self.Schedule.On(
            self.DateRules.Every(DayOfWeek.Friday),
            self.TimeRules.BeforeMarketClose(self.spy, 30),
            self.SellAtClose
        )
    
    def BuyAtOpen(self):
        if not self.Portfolio.Invested:
            self.MarketOnOpenOrder(self.spy, 100)
            self.Log("[MOO] Buy order for Monday")
    
    def SellAtClose(self):
        if self.Portfolio.Invested:
            qty = -self.Portfolio[self.spy].Quantity
            self.MarketOnCloseOrder(self.spy, qty)
            self.Log("[MOC] Sell order for Friday")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            self.Log(f"Filled: {orderEvent.FillQuantity} @ ${orderEvent.FillPrice:.2f}")

### 3.2 Limit If Touched (LIT)

Un **LIT** order devient un limit order quand le trigger price est touche.

Difference avec Stop Limit:
- **Stop Limit (achat)**: Trigger quand prix monte
- **LIT (achat)**: Trigger quand prix descend

In [ ]:
# Limit If Touched Order
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class LITExample(QCAlgorithm):
    """Acheter sur pullback avec LIT order."""
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.lit_placed = False
    
    def OnData(self, data):
        if not data.ContainsKey(self.spy) or self.lit_placed:
            return
        
        price = data[self.spy].Close
        
        # Acheter si prix descend de 2%
        trigger = price * 0.98
        limit = price * 0.985
        
        self.LimitIfTouchedOrder(self.spy, 100, trigger, limit)
        
        self.Log(f"LIT Order: Trigger=${trigger:.2f}, Limit=${limit:.2f}")
        self.lit_placed = True
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            self.Log(f"[LIT FILLED] at ${orderEvent.FillPrice:.2f}")

---
## PARTIE 4 : Order Management (25 min)

### 4.1 Order Tickets

| Propriete | Description |
|-----------|-------------|
| `OrderId` | ID unique |
| `Status` | Statut actuel |
| `AverageFillPrice` | Prix moyen d'execution |
| `Update()` | Modifier l'ordre |
| `Cancel()` | Annuler l'ordre |

In [ ]:
# Modifier un ordre avec Update
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class UpdateOrderExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 3, 31)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.ticket = None
        self.days = 0
    
    def OnData(self, data):
        if not data.ContainsKey(self.spy):
            return
        
        price = data[self.spy].Close
        
        # Placer un limit order
        if self.ticket is None and not self.Portfolio.Invested:
            limit = price * 0.98
            self.ticket = self.LimitOrder(self.spy, 100, limit)
            self.Log(f"Limit order at ${limit:.2f}")
            return
        
        # Modifier apres 2 jours
        if self.ticket and self.ticket.Status == OrderStatus.Submitted:
            self.days += 1
            
            if self.days == 2:
                # Augmenter le prix limite
                new_limit = price * 0.99
                
                update = UpdateOrderFields()
                update.LimitPrice = new_limit
                
                response = self.ticket.Update(update)
                
                if response.IsSuccess:
                    self.Log(f"Updated to ${new_limit:.2f}")
            
            elif self.days >= 5:
                # Annuler apres 5 jours
                self.ticket.Cancel()
                self.Log("Order canceled")

In [ ]:
# Annuler des ordres
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class CancelOrderExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
    
    def CancelExamples(self, ticket):
        """Differentes facons d'annuler des ordres."""
        
        # 1. Annuler un ordre specifique via ticket
        response = ticket.Cancel()
        if response.IsSuccess:
            self.Log("Order canceled via ticket")
        
        # 2. Annuler tous les ordres sur un symbol
        self.Transactions.CancelOpenOrders(self.spy)
        
        # 3. Annuler TOUS les ordres ouverts
        self.Transactions.CancelOpenOrders()
        
        # 4. Annuler par ID
        order_id = ticket.OrderId
        self.Transactions.CancelOrder(order_id)

### 4.2 OnOrderEvent Handler

`OnOrderEvent` est appele a chaque changement de statut d'un ordre.

In [ ]:
# OnOrderEvent complet avec protection automatique
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class OrderEventComplete(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        self.entry_id = None
        self.stop_id = None
        self.tp_id = None
        self.entry_price = None
    
    def OnData(self, data):
        if self.entry_id or self.Portfolio.Invested:
            return
        
        if data.ContainsKey(self.spy):
            ticket = self.MarketOrder(self.spy, 100)
            self.entry_id = ticket.OrderId
    
    def OnOrderEvent(self, orderEvent):
        oid = orderEvent.OrderId
        status = orderEvent.Status
        
        self.Log(f"Order {oid}: {status}")
        
        if status == OrderStatus.Filled:
            price = orderEvent.FillPrice
            
            # Entry rempli -> placer protection
            if oid == self.entry_id:
                self.entry_price = price
                
                # Stop-loss a 3%
                sl = self.StopMarketOrder(self.spy, -100, price * 0.97)
                self.stop_id = sl.OrderId
                
                # Take-profit a 5%
                tp = self.LimitOrder(self.spy, -100, price * 1.05)
                self.tp_id = tp.OrderId
                
                self.Log(f"Entry: ${price:.2f}, SL: ${price*0.97:.2f}, TP: ${price*1.05:.2f}")
            
            # Stop-loss touche -> annuler TP
            elif oid == self.stop_id:
                self.Transactions.CancelOpenOrders(self.spy)
                self.Log("[STOP-LOSS]")
                self.Reset()
            
            # Take-profit touche -> annuler SL
            elif oid == self.tp_id:
                self.Transactions.CancelOpenOrders(self.spy)
                self.Log("[TAKE-PROFIT]")
                self.Reset()
    
    def Reset(self):
        self.entry_id = None
        self.stop_id = None
        self.tp_id = None

---
## PARTIE 5 : Combo Orders et Avance (15 min)

### 5.1 Bracket Orders (OCO)

Un **Bracket Order** = Entry + Stop-Loss + Take-Profit avec logique OCO.

In [ ]:
# Bracket Order avec OCO
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class BracketOrderStrategy(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        self.entry_ticket = None
        self.sl_ticket = None
        self.tp_ticket = None
        self.qty = 100
    
    def OnData(self, data):
        if self.Portfolio.Invested or self.entry_ticket:
            return
        
        # Entry
        self.entry_ticket = self.MarketOrder(self.spy, self.qty)
    
    def PlaceProtection(self, entry_price):
        # Stop-loss -3%
        self.sl_ticket = self.StopMarketOrder(
            self.spy, -self.qty, entry_price * 0.97
        )
        # Take-profit +5%
        self.tp_ticket = self.LimitOrder(
            self.spy, -self.qty, entry_price * 1.05
        )
        self.Log(f"Protection: SL={entry_price*0.97:.2f}, TP={entry_price*1.05:.2f}")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status != OrderStatus.Filled:
            return
        
        oid = orderEvent.OrderId
        
        # Entry -> place protection
        if self.entry_ticket and oid == self.entry_ticket.OrderId:
            self.PlaceProtection(orderEvent.FillPrice)
        
        # SL hit -> cancel TP (OCO)
        elif self.sl_ticket and oid == self.sl_ticket.OrderId:
            if self.tp_ticket:
                self.tp_ticket.Cancel()
            self.ResetBracket()
        
        # TP hit -> cancel SL (OCO)
        elif self.tp_ticket and oid == self.tp_ticket.OrderId:
            if self.sl_ticket:
                self.sl_ticket.Cancel()
            self.ResetBracket()
    
    def ResetBracket(self):
        self.entry_ticket = None
        self.sl_ticket = None
        self.tp_ticket = None

In [ ]:
# Best Practices: Eviter les races conditions
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class BestPracticesExample(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Flags pour eviter races conditions
        self.order_pending = False
        
        # Scheduled cleanup des ordres stales
        self.Schedule.On(
            self.DateRules.EveryDay(),
            self.TimeRules.At(15, 55),
            self.CancelStaleOrders
        )
    
    def OnData(self, data):
        # MAUVAIS: peut causer des ordres en double
        # if rsi < 30:
        #     self.MarketOrder(self.spy, 100)
        
        # BON: verifier l'etat
        if self.order_pending:
            return  # Attendre que l'ordre soit traite
        
        if not self.Portfolio.Invested:
            self.MarketOrder(self.spy, 100)
            self.order_pending = True
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status in [OrderStatus.Filled, OrderStatus.Canceled]:
            self.order_pending = False
    
    def CancelStaleOrders(self):
        """Annuler les ordres de plus de 3 jours."""
        for order in self.Transactions.GetOpenOrders():
            age = (self.Time - order.CreatedTime).days
            if age > 3:
                self.Transactions.CancelOrder(order.Id)
                self.Log(f"Canceled stale order {order.Id}")

---
## PARTIE 6 : Strategie Complete (20 min)

### Strategie Breakout avec Trailing Stop

1. **Detection resistance**: Plus haut 20 jours
2. **Entry**: Stop order au-dessus de la resistance
3. **Stop-Loss**: 3% sous l'entree
4. **Take-Profit**: 8% au-dessus
5. **Trailing Stop**: Suit le prix a 2%

In [ ]:
# Strategie Breakout Complete
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class BreakoutStrategy(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.high_20 = self.MAX(self.spy, 20, Resolution.Daily)
        self.SetWarmUp(25)
        
        # Parametres
        self.breakout_buffer = 0.002
        self.stop_pct = 0.03
        self.tp_pct = 0.08
        self.trail_pct = 0.02
        self.qty = 100
        
        # Tracking
        self.entry_ticket = None
        self.sl_ticket = None
        self.tp_ticket = None
        self.entry_price = None
        self.highest = None
        
        # Stats
        self.trades = 0
        self.wins = 0
    
    def OnData(self, data):
        if self.IsWarmingUp or not self.high_20.IsReady:
            return
        
        if not data.ContainsKey(self.spy):
            return
        
        price = data[self.spy].Close
        
        # Si position ouverte, update trailing stop
        if self.Portfolio.Invested:
            self.UpdateTrailingStop(price)
            return
        
        # Si ordre en attente, ne rien faire
        if self.entry_ticket:
            return
        
        # Placer ordre breakout
        resistance = self.high_20.Current.Value
        entry = resistance * (1 + self.breakout_buffer)
        self.entry_ticket = self.StopMarketOrder(self.spy, self.qty, entry)
        self.Log(f"Breakout entry at ${entry:.2f}")
    
    def UpdateTrailingStop(self, price):
        if not self.sl_ticket:
            return
        
        if price > self.highest:
            self.highest = price
            new_stop = price * (1 - self.trail_pct)
            current_stop = self.sl_ticket.Get(OrderField.StopPrice)
            
            if new_stop > current_stop:
                update = UpdateOrderFields()
                update.StopPrice = new_stop
                if self.sl_ticket.Update(update).IsSuccess:
                    self.Log(f"Trailing stop: ${new_stop:.2f}")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status != OrderStatus.Filled:
            return
        
        oid = orderEvent.OrderId
        price = orderEvent.FillPrice
        
        # Entry
        if self.entry_ticket and oid == self.entry_ticket.OrderId:
            self.trades += 1
            self.entry_price = price
            self.highest = price
            
            self.sl_ticket = self.StopMarketOrder(
                self.spy, -self.qty, price * (1 - self.stop_pct)
            )
            self.tp_ticket = self.LimitOrder(
                self.spy, -self.qty, price * (1 + self.tp_pct)
            )
            self.Log(f"Entry: ${price:.2f}")
        
        # Stop-loss
        elif self.sl_ticket and oid == self.sl_ticket.OrderId:
            self.Transactions.CancelOpenOrders(self.spy)
            self.Log(f"[STOP-LOSS] at ${price:.2f}")
            self.Reset()
        
        # Take-profit
        elif self.tp_ticket and oid == self.tp_ticket.OrderId:
            self.wins += 1
            self.Transactions.CancelOpenOrders(self.spy)
            self.Log(f"[TAKE-PROFIT] at ${price:.2f}")
            self.Reset()
    
    def Reset(self):
        self.entry_ticket = None
        self.sl_ticket = None
        self.tp_ticket = None
    
    def OnEndOfAlgorithm(self):
        win_rate = (self.wins / self.trades * 100) if self.trades > 0 else 0
        self.Log(f"Trades: {self.trades}, Wins: {self.wins}, Win Rate: {win_rate:.1f}%")

In [ ]:
# Variante: Mean Reversion avec ordres avances
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class MeanReversionOrders(QCAlgorithm):
    """Acheter sur survente RSI avec ordres limit."""
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.rsi = self.RSI(self.spy, 14)
        self.SetWarmUp(20)
        
        self.entry_ticket = None
        self.sl_ticket = None
        self.tp_ticket = None
    
    def OnData(self, data):
        if self.IsWarmingUp or not self.rsi.IsReady:
            return
        
        if not data.ContainsKey(self.spy):
            return
        
        price = data[self.spy].Close
        rsi_val = self.rsi.Current.Value
        
        # Pas de position et RSI < 30 (survente)
        if not self.Portfolio.Invested and not self.entry_ticket:
            if rsi_val < 30:
                # Limit order pour acheter a meilleur prix
                limit = price * 0.995
                self.entry_ticket = self.LimitOrder(self.spy, 100, limit)
                self.Log(f"RSI={rsi_val:.1f}, Limit buy at ${limit:.2f}")
    
    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status != OrderStatus.Filled:
            return
        
        oid = orderEvent.OrderId
        price = orderEvent.FillPrice
        
        if self.entry_ticket and oid == self.entry_ticket.OrderId:
            # Stop-loss -2%
            self.sl_ticket = self.StopMarketOrder(self.spy, -100, price * 0.98)
            # Take-profit quand RSI > 70 simule avec limit +5%
            self.tp_ticket = self.LimitOrder(self.spy, -100, price * 1.05)
            self.Log(f"Entry at ${price:.2f}")
        
        elif self.sl_ticket and oid == self.sl_ticket.OrderId:
            self.Transactions.CancelOpenOrders(self.spy)
            self.Reset()
        
        elif self.tp_ticket and oid == self.tp_ticket.OrderId:
            self.Transactions.CancelOpenOrders(self.spy)
            self.Reset()
    
    def Reset(self):
        self.entry_ticket = None
        self.sl_ticket = None
        self.tp_ticket = None

---
## Conclusion

### Recapitulatif

| Type | Execution | Prix | Usage |
|------|-----------|------|-------|
| Market | Garantie | Non | Urgence |
| Limit | Non | Oui | Prix cible |
| Stop Market | Garantie | Non | Stop-loss |
| Stop Limit | Non | Oui | Controle slippage |
| MOO/MOC | Garantie | Non | Timing |
| LIT | Non | Oui | Pullback |

### Best Practices

1. Verifier `Portfolio.Invested` avant de passer des ordres
2. Utiliser `OnOrderEvent` pour reagir aux fills
3. Implementer OCO manuellement pour brackets
4. Gerer les partial fills et timeouts
5. Logger tous les evenements

### Ressources

- [Order Types](https://www.quantconnect.com/docs/v2/writing-algorithms/trading-and-orders/order-types)
- [Order Management](https://www.quantconnect.com/docs/v2/writing-algorithms/trading-and-orders/order-management)

**Prochain notebook**: QC-Py-10 - Techniques avancees et optimisation